In [ ]:
import utils
import os
import requests
from urllib.parse import urljoin
import time
import utils
import json
import pandas as pd
import json

### Get All Users and their SA's

In [ ]:
EXTENDED_API_URI = 'https://domino-extensions-api-svc.domino-field.svc.cluster.local'
headers = utils.generate_api_key_based_request_headers()
get_all_sa_url = urljoin(EXTENDED_API_URI, 'domino-sa-management-api/allusersa')

headers = utils.generate_api_key_based_request_headers()
response = requests.get(get_all_sa_url, headers=headers,  verify=False)
lst = response.json()['all_sa']
print(f'Total number of user based k8s sa in the domino-compute namespace = {len(lst)}')
df = pd.DataFrame(lst)
df

### Get Mutation for `user-identity-based-irsa`

In [ ]:
target_mutation_name = 'user-identity-based-irsa' 
get_mutation_url = f'{EXTENDED_API_URI}/domsed/mutation/{target_mutation_name}'
print('Get Mutations\n')
resp = requests.get(listing_url, headers=headers, verify=False)
mutations = resp.json()
json_str = json.dumps(resp.json(), indent=4)

# Print the pretty-printed JSON string
print(json_str)

## Now map SA to Domino User Id in the `user-identity-based-irsa`

In [ ]:
my_mutation = mutation['items'][0]
my_mutation
if 'managedFields' in my_mutation['metadata']:
    del my_mutation['metadata']['managedFields']
if 'resourceVersion' in my_mutation['metadata']:
    del my_mutation['metadata']['resourceVersion']


In [ ]:
for rule in my_mutation['rules']:
    if 'cloudWorkloadIdentity' in rule:
        rule['cloudWorkloadIdentity']['assume_sa_mapping']=False
        user_mappings={}
        for l in lst:
            user_mappings[l['domino_user_name']]=l['sa_name']
        rule['cloudWorkloadIdentity']['user_mappings'] = user_mappings
my_mutation


In [ ]:
## Apply mutation
publish_url = f'{EXTENDED_API_URI}/domsed/mutation/apply'
resp = requests.post(publish_url, json=my_mutation ,headers=headers, verify=False)
resp.status_code